**Please be sure to run the notebook [get-remote](get-remote.ipynb) before running this one.**

## train model

In [4]:
import mlrun, os, json
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## function

In [5]:
train_fn = mlrun.import_function('/User/functions/sklearn_classifier/function.yaml').apply(mlrun.mount_v3io())

## parameters

Please see 
**[LightGBM parameters](https://lightgbm.readthedocs.io/en/latest/Parameters.html)** for more details on all configurable LighGBM parameters.

## task

In [6]:
train_task = mlrun.NewTask(
    'train',
    params={
        # CHOOSE YOUR MODEL
        'model_pkg_class' :  "sklearn.linear_model.LogisticRegression",
        # POINT THIS TO YOUR DATA
        'data_key'        : '/User/functions/arc_to_parquet/examples/artifacts/higgs.pqt',
        'sample'          : -30_000,
        'label_column'    : "labels",
        'model_key'       : "models",
        'test_size'       : 0.05,
        'train_val_split' : 0.75,
        'rng'             : 1,
        'class_params_updates'   : {
            'random_state' : 1,
        },
        'fit_params_updates'    : {}},
    artifact_path='/User/functions/sklearn_classifier/artifacts')

tf = train_fn.run(train_task)

[mlrun] 2020-03-19 21:47:52,308 warning!, server (0.4.5) and client (0.4.6) ver dont match
[mlrun] 2020-03-19 21:47:52,308 starting run train uid=f15a52e746964360bad43309168461bc  -> http://mlrun-api:8080
[mlrun] 2020-03-19 21:47:52,465 Job is running in the background, pod: train-ks894
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignmen

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...8461bc,0,Mar 19 21:48:05,error,train,host=train-ks894kind=jobowner=admin,,class_params_updates={'random_state': 1}data_key=/User/functions/arc_to_parquet/examples/artifacts/higgs.pqtfit_params_updates={}label_column=labelsmodel_key=modelsmodel_pkg_class=sklearn.linear_model.LogisticRegressionrng=1sample=-30000test_size=0.05train_val_split=0.75,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run f15a52e746964360bad43309168461bc  , !mlrun logs f15a52e746964360bad43309168461bc 
[mlrun] 2020-03-19 21:48:21,844 run executed, status=error
runtime error: error, check logs


RunError: error, check logs

## tests

In [ ]:
# refactor: take a simple make_classification dataset...

In [ ]:
outkeys = list(tf.outputs.keys())

assert outkeys == ['accuracy',
                   'avg_precscore',
                   'f1_score',
                   'rocauc',
                   'test_set',
                   'model',
                   'roc',
                   'history',
                   'confusion',
                   'featimp.png',
                   'featimp.csv']